In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train.head()

In [ ]:
resize_img = '../input/resized-plant2021/img_sz_512'
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

In [ ]:
path = './test_img_resize'
if not os.path.isdir(path):
    os.makedirs(path)

In [ ]:
img_counts = submission.shape[0]

for i in range(img_counts):
    img = cv2.imread('../input/plant-pathology-2021-fgvc8/test_images/' + submission.loc[i,'image'])
    img = cv2.resize(img,(640,640))
    #cv2.imshow('img'+str(i),img)
    b,g,r = cv2.split(img)

    kernel_size = 5
    blur = cv2.GaussianBlur(g,(kernel_size, kernel_size), 0)
    blur = cv2.GaussianBlur(blur,(kernel_size, kernel_size), 0)
    
    edges = cv2.Canny(blur, 30, 200)
    #cv2.imshow('edges'+str(i),edges)
    edge_box = []
    for x in range(edges.shape[0]):
        for y in range(edges.shape[1]):
            if edges[x][y] != 0:
                edge_box.append((x, y))
                
    row_min = edge_box[np.argsort([box[0] for box in edge_box])[0]][0]
    row_max = edge_box[np.argsort([box[0] for box in edge_box])[-1]][0]
    col_min = edge_box[np.argsort([box[1] for box in edge_box])[0]][1]
    col_max = edge_box[np.argsort([box[1] for box in edge_box])[-1]][1]

    test_img= img[row_min:row_max, col_min:col_max]
    #cv2.imshow('cut'+str(i),new_img)
    test_img_resize = cv2.resize(test_img,(512,512))
    
    path = './test_img_resize'
    if not os.path.isdir(path):
        os.makedirs(path)
    
    cv2.imwrite( './test_img_resize/' + submission.loc[i,'image'],test_img_resize)